In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from pathlib import Path

In [9]:
ROOT_DIR = Path.cwd().parent 

ruta_datos = ROOT_DIR / "data" / "spotify_songs.csv"

df = pd.read_csv(ruta_datos)